In [31]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import pandas as pd
import numpy as np
import os, glob, shutil

In [78]:
new_num_df = pd.read_excel('New_number.xlsx')
translation_dict = {old: new for new, old in zip(new_num_df['New'], new_num_df['Old'])}
old_space_df = pd.read_csv(r'ignored\ConnectToHTE\Data\full_space.csv')
old_space_binol = np.unique(old_space_df['Binol'].to_numpy())
old_space_ligand = np.unique(old_space_df['Ligand'].to_numpy())
len(old_space_ligand), len(old_space_binol)

(178, 44)

In [79]:
assert np.all(np.isin(old_space_binol, new_num_df['Old']))
assert np.all(np.isin(old_space_ligand, new_num_df['Old']))


In [ ]:
data_csv = pd.read_csv("ignored/backup/Data_clear.csv")
data_csv['Index'] = data_csv['Index'].map(translation_dict)
data_csv.drop(columns=['Type_Index'], inplace=True)
data_csv = data_csv.dropna().reset_index(drop=True)
data_csv['Index'] = data_csv['Index'].astype(int)
data_csv.sort_values(by=['Index'], inplace=True)
# data_csv.to_csv('Data_clear.csv', index=False)

In [ ]:
for row_id, row in new_num_df.iterrows():
    new, old = row['New'], row["Old"]
    for subdir in ['GS_OPT', 'Mols', "newGS", 'newmols']:
        files = glob.glob(f"ignored/backup/{subdir}/{old:05}*")
        for each in files:
            new_file = os.path.join(f'Data/DFT/{subdir}', os.path.basename(each).replace(f'{old:05}', f'{new:05}'))
            # shutil.copy(each, new_file)

In [ ]:
for row_id, row in new_num_df.iterrows():
    new, old = row['New'], row["Old"]
    paths = glob.glob(f"ignored/backup/Mol_xtb/*/{old:05}*")
    for each in paths[:0]:
        new_path = os.path.join(f'Data/DFT/Mol_xtb/charge_0_0', os.path.basename(each).replace(f'{old:05}', f'{new:05}'))
        # shutil.copytree(each, new_path)

In [112]:
import pickle
with open(r"Data/all_fp_map2.pkl", 'rb')as f:
    qm_dict, area_dict = pickle.load(f)

In [114]:
new_qm_dict = {row['New']: qm_dict[row['Old']] for _, row in new_num_df.iterrows()}
new_area_dict = {row['New']: area_dict[row['Old']] for _, row in new_num_df.iterrows()}

In [ ]:
# with open(r"Data/all_fp_map.pkl", 'wb')as f:
#     pickle.dump([new_qm_dict, new_area_dict], f)

In [192]:
for csv in glob.glob("Data/*/*.csv") + glob.glob("Data/*.csv"):
    df = pd.read_csv(csv)
    # print(len(df))
    for name in ['Binol_smiles', 'Ligand_smiles', 'Binol_conf_id', 'Ligand_conf_id', 'Binol_Sites', 'Ligand_Sites', 'Binol_G', 'Ligand_G', 'Unnamed: 4']:
        if name in df.columns:
            df = df.drop(columns = name)
    # df["Binol"] = df["Binol"].map(translation_dict)
    # df["Ligand"] = df["Ligand"].map(translation_dict)
    # df = df.dropna().reset_index(drop=True)
    df['Binol'] = df['Binol'].astype(int)
    df['Ligand'] = df['Ligand'].astype(int)
    df.to_csv(csv, index=False)

In [ ]:
for csv in glob.glob("Data/Iteration_2/Result/*.csv"):

In [152]:
for id_ in range(1,16):
    csv = f"Data/Iteration_2/Result_/BINOL_result_{id_:04}.csv"
    new_csv = f"Data/Iteration_2/Result_/BINOL_result_{id_ - 1:04}.csv"
    shutil.move(csv, new_csv)

In [178]:
for id_ in range(15):
    csv = f"Data/Iteration_2/Result/BINOL_result_{id_:04}.csv"
    newcsv = f"Data/Iteration_2/Iteration_Data/iter_{id_:05}.csv"
    df = pd.read_csv(csv)
    df.drop(columns=['R', 'S'], inplace=True, errors='ignore')
    df.to_csv(newcsv, index=False)

In [181]:
AL_data = pd.read_csv("Data/Iteration_2/Full_Space_ActiveLearning.csv")
AL_id = [f'{row.Binol:05}_{row.Ligand:05}' for _, row in AL_data.iterrows()]


In [182]:
for id_ in range(15):
    csv = f"Data/Iteration_2/Iteration_Data/iter_{id_:05}.csv"
    df = pd.read_csv(csv)
    temp_id = [f'{int(row.Binol):05}_{int(row.Ligand):05}' for _, row in df.iterrows()]
    collect_id = [AL_id.index(t_id) for t_id in temp_id]
    print(len(collect_id))
    np.save(f"Data/Iteration_2/Iteration_Data/iter_{id_:05}.npy", np.array(collect_id))

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
